In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, cross_val_score, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from datetime import datetime

In [2]:
df = pd.read_csv("fine_food_dataset.csv")

In [3]:
df.shape

(40000, 7)

In [4]:
df.columns

Index(['Unnamed: 0', 'Text', 'Score', 'sentiment',
       'review_after_removing_html_tags', 'review_after_removing_punctuation',
       'review_after_pos_and_lemmatization'],
      dtype='object')

In [5]:
df.isna().sum()

Unnamed: 0                            0
Text                                  0
Score                                 0
sentiment                             0
review_after_removing_html_tags       0
review_after_removing_punctuation     0
review_after_pos_and_lemmatization    0
dtype: int64

In [6]:
df[ df.sentiment == 1 ].shape

(20000, 7)

In [7]:
df[ df.sentiment == 0 ].shape

(20000, 7)

In [8]:
bag_of_word_model = CountVectorizer(analyzer='word',ngram_range=(1,3))

X = bag_of_word_model.fit_transform(df["review_after_pos_and_lemmatization"])
y = df["sentiment"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
y_train[ y_train == 1 ].shape

(16039,)

In [11]:
y_train[ y_train == 0 ].shape

(15961,)

In [12]:
y_test[ y_test == 1 ].shape

(3961,)

In [13]:
y_test[ y_test == 0 ].shape

(4039,)

#### KFold cross-validation

In [14]:
# Set up k-fold cross-validation
kf = KFold(n_splits=4, shuffle=True)

random_forest_scores = []
logistic_scores = []
svm_scores = []
gd_boost_scores = []

# Generate indices for each fold
for train_index, test_index in kf.split(X):
    x_train_data, x_test_data = X[train_index], X[test_index]
    y_train_data, y_test_data = y[train_index], y[test_index]

    ## Random Forest
    model_forest = RandomForestClassifier(max_depth=10, n_estimators=500)

    print("Fitting random forest to training data....")    
    model_forest = model_forest.fit(x_train_data, y_train_data)
    
    y_pred_data = model_forest.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    random_forest_scores.append(score)
    print(random_forest_scores)
    
    ## Logistic Regression
    
    model_logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)
    print("Fitting Logistic Regression to training data....")  
    model_logistic_regression = model_logistic_regression.fit(x_train_data, y_train_data)
    
    y_pred_data = model_logistic_regression.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    logistic_scores.append(score)
    print(logistic_scores)
    
    ## SVM
    model_svm = SVC(C=0.1, kernel='linear')
    print("Fitting SVM to training data....")  
    model_svm = model_svm.fit(x_train_data, y_train_data)
    
    y_pred_data = model_svm.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    svm_scores.append(score)
    print(svm_scores)
    
    ## Gradient Boosting
    gb_classifier = GradientBoostingClassifier(random_state=2)
    print("Fitting Gradient Boosting to training data....")    
    gb_classifier.fit(x_train_data, y_train_data)
    
    y_pred_data = gb_classifier.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    gd_boost_scores.append(score)
    print(gd_boost_scores)
    
    print("=======================================================")

# print(f"Cross-validation scores: {scores}")
# print(f"Average score: {np.mean(scores)}")

Fitting random forest to training data....
[0.8234]
Fitting Logistic Regression to training data....
[0.8624]
Fitting SVM to training data....
[0.895]
Fitting Gradient Boosting to training data....
[0.7911]
Fitting random forest to training data....
[0.8234, 0.8276]
Fitting Logistic Regression to training data....
[0.8624, 0.8559]
Fitting SVM to training data....
[0.895, 0.8877]
Fitting Gradient Boosting to training data....
[0.7911, 0.7835]
Fitting random forest to training data....
[0.8234, 0.8276, 0.8093]
Fitting Logistic Regression to training data....
[0.8624, 0.8559, 0.8585]
Fitting SVM to training data....
[0.895, 0.8877, 0.8878]
Fitting Gradient Boosting to training data....
[0.7911, 0.7835, 0.7914]
Fitting random forest to training data....
[0.8234, 0.8276, 0.8093, 0.8421]
Fitting Logistic Regression to training data....
[0.8624, 0.8559, 0.8585, 0.8591]
Fitting SVM to training data....
[0.895, 0.8877, 0.8878, 0.8917]
Fitting Gradient Boosting to training data....
[0.7911, 0.78

In [15]:
print("Random Forest KFold cross-validation")
print(f"Cross-validation scores: {random_forest_scores}")
print(f"Average score: {np.mean(random_forest_scores)}\n")
print("Logistic Regression KFold cross-validation")
print(f"Cross-validation scores: {logistic_scores}")
print(f"Average score: {np.mean(logistic_scores)}\n")
print("SVM cross-validation")
print(f"Cross-validation scores: {svm_scores}")
print(f"Average score: {np.mean(svm_scores)}\n")
print("Gradient Boosting KFold cross-validation")
print(f"Cross-validation scores: {gd_boost_scores}")
print(f"Average score: {np.mean(gd_boost_scores)}\n")

Random Forest KFold cross-validation
Cross-validation scores: [0.8234, 0.8276, 0.8093, 0.8421]
Average score: 0.8256

Logistic Regression KFold cross-validation
Cross-validation scores: [0.8624, 0.8559, 0.8585, 0.8591]
Average score: 0.858975

SVM cross-validation
Cross-validation scores: [0.895, 0.8877, 0.8878, 0.8917]
Average score: 0.8905500000000001

Gradient Boosting KFold cross-validation
Cross-validation scores: [0.7911, 0.7835, 0.7914, 0.7912]
Average score: 0.7893



#### Random Forest

In [16]:
# Start the timer
start_time = datetime.now()

model_forest = RandomForestClassifier(max_depth=10, n_estimators=500)

print("Fitting random forest to training data....")    
model_forest = model_forest.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Random Forest Training Runtime:', runtime)

Fitting random forest to training data....
Random Forest Training Runtime: 0:03:30.756621


In [17]:
model_forest.score(X_test, y_test)

0.813125

In [18]:
model_forest.score(X_train, y_train)

0.837625

In [19]:
# Start the timer
start_time = datetime.now()

y_pred = model_forest.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Random Forest Prediction Runtime:', runtime)

Random Forest Prediction Runtime: 0:00:11.988904


In [20]:
y_pred = cross_val_predict(model_forest, X_test, y_test)

In [21]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.95      0.82      4039
           1       0.92      0.64      0.76      3961

    accuracy                           0.80      8000
   macro avg       0.83      0.79      0.79      8000
weighted avg       0.83      0.80      0.79      8000



In [22]:
print(confusion_matrix(y_test, y_pred))

[[3827  212]
 [1422 2539]]


In [23]:
filename = 'models/rd_ngram_ml_model.sav'
pickle.dump(model_forest, open(filename, 'wb'))

#### Logistic Regression

In [24]:
# Start the timer
start_time = datetime.now()

model_logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)

print("Fitting Logistic Regression to training data....")  
model_logistic_regression = model_logistic_regression.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Logistic Regression Training Runtime:', runtime)

Fitting Logistic Regression to training data....
Logistic Regression Training Runtime: 0:00:04.192893


In [25]:
model_logistic_regression.score(X_test, y_test)

0.86725

In [26]:
model_logistic_regression.score(X_train, y_train)

0.8746875

In [27]:
# Start the timer
start_time = datetime.now()

y_pred = model_logistic_regression.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Logistic Regression Prediction Runtime:', runtime)

Logistic Regression Prediction Runtime: 0:00:00.015583


In [28]:
y_pred = cross_val_predict(model_logistic_regression, X_test, y_test)

In [29]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.82      0.82      4039
           1       0.82      0.81      0.82      3961

    accuracy                           0.82      8000
   macro avg       0.82      0.82      0.82      8000
weighted avg       0.82      0.82      0.82      8000



In [30]:
print(confusion_matrix(y_test, y_pred))

[[3313  726]
 [ 734 3227]]


In [31]:
filename = 'models/lg_ngram_ml_model.sav'
pickle.dump(model_logistic_regression, open(filename, 'wb'))

#### SVM

In [32]:
# Start the timer
start_time = datetime.now()

model_svm = SVC(C=0.1, kernel='linear')

model_svm = model_svm.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:25:53.180418


In [33]:
model_svm.score(X_test, y_test)

0.898375

In [34]:
model_svm.score(X_train, y_train)

0.99996875

In [35]:
# Start the timer
start_time = datetime.now()

y_pred = model_svm.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:01:17.993960


In [36]:
y_pred = cross_val_predict(model_svm, X_test, y_test)

In [37]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.85      0.86      4039
           1       0.85      0.85      0.85      3961

    accuracy                           0.85      8000
   macro avg       0.85      0.85      0.85      8000
weighted avg       0.85      0.85      0.85      8000



In [38]:
print(confusion_matrix(y_test, y_pred))

[[3451  588]
 [ 579 3382]]


In [39]:
filename = 'models/svm_ngram_ml_model.sav'
pickle.dump(model_svm, open(filename, 'wb'))

#### Gradient Boosting Classifier

In [40]:
# Start the timer
start_time = datetime.now()


# create a Gradient Boosting classifier with default hyperparameters
gb_classifier = GradientBoostingClassifier(random_state=2)

# fit the classifier to the training data
gb_classifier.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Gradient Boosting Prediction Runtime:', runtime)


Gradient Boosting Prediction Runtime: 2:29:08.895089


In [41]:
gb_classifier.score(X_test, y_test)

0.796375

In [42]:
gb_classifier.score(X_train, y_train)

0.79840625

In [43]:
# Start the timer
start_time = datetime.now()

y_pred = gb_classifier.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:00:00.029219


In [44]:
y_pred = cross_val_predict(gb_classifier, X_test, y_test)

In [45]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.80      0.79      4039
           1       0.79      0.77      0.78      3961

    accuracy                           0.78      8000
   macro avg       0.78      0.78      0.78      8000
weighted avg       0.78      0.78      0.78      8000



In [46]:
filename = 'models/gb_boosting_ngram_ml_model.sav'
pickle.dump(gb_classifier, open(filename, 'wb'))